[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RizanSM/zero_shot_llms_in_HIL_RL/blob/main/01_Code/01_Highway_Env/02_Default_Environment/01_HF_DIRECT/03_Model_testing_HF_D_Default_highway_env.ipynb)

In [ ]:
# Install the required libraries in your Google Colab environment
!pip install gymnasium stable-baselines3 highway-env -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.2 MB/s eta 0:00:00


In [ ]:
# Import the necessary libraries
import gymnasium as gym
import highway_env
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from stable_baselines3 import PPO
from google.colab import drive
from google.colab import data_table

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


CALCULATE MISALIGNMENT(FMA) FUNCTION

In [ ]:
def calculate_misalignment(df, reward_threshold=0, lane_change_penalty=1, collision_penalty=5, reward_penalty=2):
    """
    Calculate misalignment score for each episode in the dataset.

    Parameters:
        df (pd.DataFrame): The data frame containing highway environment trajectories.
        reward_threshold (float): Threshold below which reward is considered suboptimal.
        lane_change_penalty (float): Penalty factor for unnecessary lane changes.
        collision_penalty (float): Penalty factor for collisions.
        reward_penalty (float): Penalty factor for suboptimal rewards.

    Returns:
        pd.DataFrame: Data frame with misalignment scores per episode.
    """
    misalignment_scores = []  # Changed to 'misalignment_scores' to store all episode scores

    for episode in df['episode'].unique():
        episode_df = df[df['episode'] == episode].sort_values(by='time_step')

        # Calculate lane changes
        lane_changes = episode_df['lane_index'].diff().abs().sum()

        # Count collisions
        collisions = episode_df['collision_flag'].sum()

        # Count suboptimal rewards
        suboptimal_rewards = (episode_df['reward'] < reward_threshold).sum()

        # Compute misalignment score
        misalignment_score = (
            lane_change_penalty * lane_changes +
            collision_penalty * collisions +
            reward_penalty * suboptimal_rewards
        )

        misalignment_scores.append({'episode': episode, 'Misalignment Score': misalignment_score})

    df = pd.DataFrame(misalignment_scores) # Create the DataFrame outside the loop

    # Calculate the overall misalignment as the mean of per-episode misalignment
    Average_misalignment = df['Misalignment Score'].mean() # Calculate mean of 'Misalignment Score' column

    return df, Average_misalignment

AVERAGE TERMINATE TIME (ATT) FUNCTION

In [ ]:
# Function to calculate the collision rate and return a DataFrame
def calculate_collision_rate(test_trajectory_df):
    # Initialize an empty list to store the results
    results = []

    # Group by 'episode' to process each episode separately
    episodes = test_trajectory_df.groupby('episode')

    # Iterate through each episode
    for episode, episode_data in episodes:
        # Count the number of collisions in the current episode
        num_collisions = episode_data['collision_flag'].sum()

        # Get the total number of timesteps in the current episode
        total_steps = len(episode_data)

        # Append the results to the list
        results.append({
            'Episode': episode,
            'Total Steps': total_steps
        })

    # Convert the list of results to a DataFrame
    results_df = pd.DataFrame(results)

    # Calculate the overall Collision rate as the mean of per-episode Collision rate
    Average_terminate_time = results_df['Total Steps'].mean()

    # Return the resulting DataFrame
    return results_df, Average_terminate_time

STABILITY OF THE LEARNING CURVES OF THE MODEL FUNCTION

In [ ]:
def analyze_learning_curve(test_trajectory_df):
    # Step 1: Calculate Cumulative Reward Per Episode
    cumulative_rewards = test_trajectory_df.groupby('episode')['reward'].sum().reset_index()
    cumulative_rewards.columns = ['episode', 'cumulative_reward']

    # Step 2: Calculate Smoothed Rewards using Moving Average (window size of 10 episodes)
    window_size = 10
    cumulative_rewards['smoothed_reward'] = cumulative_rewards['cumulative_reward'].rolling(window=window_size, min_periods=1).mean()

    # Step 3: Calculate Variance and Standard Deviation of Cumulative Rewards
    variance = cumulative_rewards['cumulative_reward'].var()
    std_dev = cumulative_rewards['cumulative_reward'].std()
    final_reward = cumulative_rewards['cumulative_reward'].mean()

    # Step 4: Return the dataframe containing all results
    return cumulative_rewards, variance, std_dev, final_reward

COMBINING ALL THE DATA FRAMES

In [ ]:
def create_cummulative_reward(df1, df2, df3, df4, df5):
    """
    Create a data frame 'ideal_cummulative_reward' that contains:
      - 'episode' column (common across all data frames)
      - 'cumulative_reward_1' to 'cumulative_reward_5' columns from each data frame respectively,
        where each input data frame has columns 'episode' and 'cummulative_reward'.
      - 'mean_cumulative_reward' column containing the row-wise mean of the 5 cumulative rewards.

    Parameters:
      df1, df2, df3, df4, df5 (pd.DataFrame): Data frames with columns 'episode' and 'cummulative_reward'.

    Returns:
      pd.DataFrame: The merged and aggregated data frame.
    """
    # Rename the 'cummulative_reward' column in each data frame to a unique name.
    df1_renamed = df1.rename(columns={'cumulative_reward': 'cumulative_reward_1'})
    df2_renamed = df2.rename(columns={'cumulative_reward': 'cumulative_reward_2'})
    df3_renamed = df3.rename(columns={'cumulative_reward': 'cumulative_reward_3'})
    df4_renamed = df4.rename(columns={'cumulative_reward': 'cumulative_reward_4'})
    df5_renamed = df5.rename(columns={'cumulative_reward': 'cumulative_reward_5'})

    # Merge the data frames on the 'episode' column.
    merged_df = df1_renamed[['episode', 'cumulative_reward_1']].copy()
    merged_df = merged_df.merge(df2_renamed[['episode', 'cumulative_reward_2']], on='episode')
    merged_df = merged_df.merge(df3_renamed[['episode', 'cumulative_reward_3']], on='episode')
    merged_df = merged_df.merge(df4_renamed[['episode', 'cumulative_reward_4']], on='episode')
    merged_df = merged_df.merge(df5_renamed[['episode', 'cumulative_reward_5']], on='episode')

    # Compute the episode-wise mean of the cumulative rewards.
    reward_columns = [
        'cumulative_reward_1',
        'cumulative_reward_2',
        'cumulative_reward_3',
        'cumulative_reward_4',
        'cumulative_reward_5'
    ]
    merged_df['mean_cumulative_reward'] = merged_df[reward_columns].mean(axis=1)

    return merged_df

CALCULATE FINAL MISALIGNMENT, AVERAGE TERMINATE TIME & AVERAGE EPISODIC REWARD

In [ ]:
def calculate_performance(value_1, value_2, value_3):
    mean_1 = sum(value_1) / len(value_1)
    mean_2 = sum(value_2) / len(value_2)
    mean_3 = sum(value_3) / len(value_3)

    return mean_1, mean_2, mean_3


0. MODEL TESTING (HF_D_Ideal)

In [ ]:
trajectory_df_1 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/1_hf_d_ideal/1_hf_d_ideal_df.pkl')            # Update directory location 1
trajectory_df_2 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/1_hf_d_ideal/2_hf_d_ideal_df.pkl')            # Update directory location 2
trajectory_df_3 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/1_hf_d_ideal/3_hf_d_ideal_df.pkl')            # Update directory location 3
trajectory_df_4 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/1_hf_d_ideal/4_hf_d_ideal_df.pkl')            # Update directory location 4
trajectory_df_5 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/1_hf_d_ideal/5_hf_d_ideal_df.pkl')            # Update directory location 5

TESTING THE MODEL BY ANALYZING THE DATA FRAME <br>



1. Human feedback Direct Ideal data frame 1<br>
trajectory_df_1 ---> 1_hf_d_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_1)

Output hidden; open in https://colab.research.google.com to view.

A. MISALIGNMENT FOR DATA FRAME (trajectory_df_1)

In [ ]:
misalignment_df_1, misalignment_1 = calculate_misalignment(trajectory_df_1)
print("misalignment:", misalignment_1)

misalignment: 0.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_1)

,episode,Misalignment Score
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
95,95,0.0
96,96,0.0
97,97,0.0
98,98,0.0


B. AVERAGE TERMINATE TIME FOR DATA FRAME (trajectory_df_1)

In [ ]:
collision_rate_df_1, collision_rate_1 = calculate_collision_rate(trajectory_df_1)
print("average terminate time:", collision_rate_1)

average terminate time: 40.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_1)

,Episode,Total Steps
0,0,40
1,1,40
2,2,40
3,3,40
4,4,40
...,...,...
95,95,40
96,96,40
97,97,40
98,98,40


C. STABILITY OF LEARNING CURVES (trajectory_df_1)

In [ ]:
learning_curve_result_df_1, variance_1, std_dev_1, final_reward_1 = analyze_learning_curve(trajectory_df_1)
print("Variance:", variance_1)
print("Standard Deviation:", std_dev_1)
print("average episodic reward:", final_reward_1)

Variance: 4.130762558248207e-27
Standard Deviation: 6.427100869169712e-14
average episodic reward: 29.360814668060115


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_1)

,episode,cumulative_reward,smoothed_reward
0,0,29.360815,29.360815
1,1,29.360815,29.360815
2,2,29.360815,29.360815
3,3,29.360815,29.360815
4,4,29.360815,29.360815
...,...,...,...
95,95,29.360815,29.360815
96,96,29.360815,29.360815
97,97,29.360815,29.360815
98,98,29.360815,29.360815


2. Human feedback Direct Ideal data frame 2<br>
trajectory_df_2 ---> 2_hf_d_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_2)

Output hidden; open in https://colab.research.google.com to view.

A. MISALIGNMENT FOR DATA FRAME (trajectory_df_2)

In [ ]:
misalignment_df_2, misalignment_2 = calculate_misalignment(trajectory_df_2)
print("misalignment:", misalignment_2)

misalignment: 0.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_2)

,episode,Misalignment Score
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
95,95,0.0
96,96,0.0
97,97,0.0
98,98,0.0


B. AVERAGE TERMINATE TIME FOR DATA FRAME (trajectory_df_2)

In [ ]:
collision_rate_df_2, collision_rate_2 = calculate_collision_rate(trajectory_df_2)
print("average terminate time:", collision_rate_2)

average terminate time: 40.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_2)

,Episode,Total Steps
0,0,40
1,1,40
2,2,40
3,3,40
4,4,40
...,...,...
95,95,40
96,96,40
97,97,40
98,98,40


C. STABILITY OF LEARNING CURVES (trajectory_df_2)

In [ ]:
learning_curve_result_df_2, variance_2, std_dev_2, final_reward_2 = analyze_learning_curve(trajectory_df_2)
print("Variance:", variance_2)
print("Standard Deviation:", std_dev_2)
print("average episodic reward:", final_reward_2)

Variance: 4.130762558248207e-27
Standard Deviation: 6.427100869169712e-14
average episodic reward: 29.360814668060115


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_2)

,episode,cumulative_reward,smoothed_reward
0,0,29.360815,29.360815
1,1,29.360815,29.360815
2,2,29.360815,29.360815
3,3,29.360815,29.360815
4,4,29.360815,29.360815
...,...,...,...
95,95,29.360815,29.360815
96,96,29.360815,29.360815
97,97,29.360815,29.360815
98,98,29.360815,29.360815


3. Human feedback Direct Ideal data frame 3<br>
trajectory_df_3 ---> 3_hf_d_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_3)

Output hidden; open in https://colab.research.google.com to view.

A. MISALIGNMENT FOR DATA FRAME (trajectory_df_3)

In [ ]:
misalignment_df_3, misalignment_3 = calculate_misalignment(trajectory_df_3)
print("misalignment:", misalignment_3)

misalignment: 0.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_3)

,episode,Misalignment Score
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
95,95,0.0
96,96,0.0
97,97,0.0
98,98,0.0


B. COLLISION RATE FOR DATA FRAME (trajectory_df_3)

In [ ]:
collision_rate_df_3, collision_rate_3 = calculate_collision_rate(trajectory_df_3)
print("average terminate time:", collision_rate_3)

average terminate time: 40.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_3)

,Episode,Total Steps
0,0,40
1,1,40
2,2,40
3,3,40
4,4,40
...,...,...
95,95,40
96,96,40
97,97,40
98,98,40


C. STABILITY OF LEARNING CURVES (trajectory_df_3)

In [ ]:
learning_curve_result_df_3, variance_3, std_dev_3, final_reward_3 = analyze_learning_curve(trajectory_df_3)
print("Variance:", variance_3)
print("Standard Deviation:", std_dev_3)
print("average episodic reward:", final_reward_3)

Variance: 6.247140905992659e-28
Standard Deviation: 2.4994281157882216e-14
average episodic reward: 27.58303689028236


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_3)

,episode,cumulative_reward,smoothed_reward
0,0,27.583037,27.583037
1,1,27.583037,27.583037
2,2,27.583037,27.583037
3,3,27.583037,27.583037
4,4,27.583037,27.583037
...,...,...,...
95,95,27.583037,27.583037
96,96,27.583037,27.583037
97,97,27.583037,27.583037
98,98,27.583037,27.583037


4. Human feedback Direct Ideal data frame 4<br>
trajectory_df_4 ---> 4_hf_d_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_4)

Output hidden; open in https://colab.research.google.com to view.

A. MISALIGNMENT FOR DATA FRAME (trajectory_df_4)

In [ ]:
misalignment_df_4, misalignment_4 = calculate_misalignment(trajectory_df_4)
print("misalignment:", misalignment_4)

misalignment: 0.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_4)

,episode,Misalignment Score
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
95,95,0.0
96,96,0.0
97,97,0.0
98,98,0.0


B. COLLISION RATE FOR DATA FRAME (trajectory_df_4)

In [ ]:
collision_rate_df_4, collision_rate_4 = calculate_collision_rate(trajectory_df_4)
print("average terminate time:", collision_rate_4)

average terminate time: 40.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_4)

,Episode,Total Steps
0,0,40
1,1,40
2,2,40
3,3,40
4,4,40
...,...,...
95,95,40
96,96,40
97,97,40
98,98,40


C. STABILITY OF LEARNING CURVES (trajectory_df_4)

In [ ]:
learning_curve_result_df_4, variance_4, std_dev_4, final_reward_4 = analyze_learning_curve(trajectory_df_4)
print("Variance:", variance_4)
print("Standard Deviation:", std_dev_4)
print("average episodic reward:", final_reward_4)

Variance: 4.130762558248207e-27
Standard Deviation: 6.427100869169712e-14
average episodic reward: 29.360814668060115


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_4)

,episode,cumulative_reward,smoothed_reward
0,0,29.360815,29.360815
1,1,29.360815,29.360815
2,2,29.360815,29.360815
3,3,29.360815,29.360815
4,4,29.360815,29.360815
...,...,...,...
95,95,29.360815,29.360815
96,96,29.360815,29.360815
97,97,29.360815,29.360815
98,98,29.360815,29.360815


5. Human feedback Direct Ideal data frame 5<br>
trajectory_df_5 ---> 5_hf_d_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_5)

Output hidden; open in https://colab.research.google.com to view.

A. MISALIGNMENT FOR DATA FRAME (trajectory_df_5)

In [ ]:
misalignment_df_5, misalignment_5 = calculate_misalignment(trajectory_df_5)
print("misalignment:", misalignment_5)

misalignment: 0.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_5)

,episode,Misalignment Score
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
95,95,0.0
96,96,0.0
97,97,0.0
98,98,0.0


B. COLLISION RATE FOR DATA FRAME (trajectory_df_5)

In [ ]:
collision_rate_df_5, collision_rate_5 = calculate_collision_rate(trajectory_df_5)
print("average terminate time:", collision_rate_5)

average terminate time: 40.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_5)

,Episode,Total Steps
0,0,40
1,1,40
2,2,40
3,3,40
4,4,40
...,...,...
95,95,40
96,96,40
97,97,40
98,98,40


C. STABILITY OF LEARNING CURVES (trajectory_df_5)

In [ ]:
learning_curve_result_df_5, variance_5, std_dev_5, final_reward_5 = analyze_learning_curve(trajectory_df_5)
print("Variance:", variance_5)
print("Standard Deviation:", std_dev_5)
print("average episodic reward:", final_reward_5)

Variance: 1.274926715508706e-27
Standard Deviation: 3.5706115939831735e-14
average episodic reward: 26.694148001393447


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_5)

,episode,cumulative_reward,smoothed_reward
0,0,26.694148,26.694148
1,1,26.694148,26.694148
2,2,26.694148,26.694148
3,3,26.694148,26.694148
4,4,26.694148,26.694148
...,...,...,...
95,95,26.694148,26.694148
96,96,26.694148,26.694148
97,97,26.694148,26.694148
98,98,26.694148,26.694148


COMBINED DATA FRAMES - HF-D-IDEAL

In [ ]:
cummulative_reward = create_cummulative_reward(
        learning_curve_result_df_1,
        learning_curve_result_df_2,
        learning_curve_result_df_3,
        learning_curve_result_df_4,
        learning_curve_result_df_5
    )

data_table.enable_dataframe_formatter()
data_table.DataTable(cummulative_reward)

,episode,cumulative_reward_1,cumulative_reward_2,cumulative_reward_3,cumulative_reward_4,cumulative_reward_5,mean_cumulative_reward
0,0,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926
1,1,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926
2,2,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926
3,3,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926
4,4,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926
...,...,...,...,...,...,...,...
95,95,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926
96,96,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926
97,97,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926
98,98,29.360815,29.360815,27.583037,29.360815,26.694148,28.471926


FINAL MISALIGNMENT, AVERAGE TERMINATE TIME & AVERAGE EPISODIC REWARD : HF-D-IDEAL

In [ ]:
 # Organize the values into lists
misalignment_list_1 = [misalignment_1, misalignment_2, misalignment_3, misalignment_4, misalignment_5]
average_terminate_time_list_1 = [collision_rate_1, collision_rate_2, collision_rate_3, collision_rate_4, collision_rate_5]
average_episodic_reward_list_1 = [final_reward_1, final_reward_2, final_reward_3, final_reward_4, final_reward_5]

# Call the function to compute the mean performance values
mean_misalignment_1, mean_average_terminate_time_1 , mean_average_episodic_reward_1 = calculate_performance(misalignment_list_1, average_terminate_time_list_1, average_episodic_reward_list_1)

# Print the results
print("HF_D_IDEAL FINAL MISALIGNMENT:", mean_misalignment_1)
print("HF_D_IDEAL AVERAGE TERMINATE TIME:", mean_average_terminate_time_1)
print("HF_D_IDEAL AVERAGE EPISODIC REWARD:", mean_average_episodic_reward_1)

HF_D_IDEAL FINAL MISALIGNMENT: 0.0
HF_D_IDEAL AVERAGE TERMINATE TIME: 40.0
HF_D_IDEAL AVERAGE EPISODIC REWARD: 28.471925779171233
